In [2]:
"""
Análisis Completo: Tickers Activos + Inactivos
Snapshot: 2025-10-24
"""
import polars as pl
from pathlib import Path
import os

# Configurar
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
os.chdir(project_root)

# =============================================================================
# CARGAR RESUMEN CSV
# =============================================================================
csv_path = "temp_active_counts_complete.csv"
df_summary = pl.read_csv(csv_path)

print("="*100)
print("RESUMEN GENERAL")
print("="*100)
display(df_summary)
print()

# =============================================================================
# CARGAR SNAPSHOT COMPLETO
# =============================================================================
snapshot_path = "raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-24/tickers_all.parquet"
df_all = pl.read_parquet(snapshot_path)

print(f"TOTAL TICKERS: {len(df_all):,}")
print()

# Separar activos e inactivos
df_active = df_all.filter(pl.col("active") == True)
df_inactive = df_all.filter(pl.col("active") == False)

print(f"ACTIVOS:   {len(df_active):>8,} ({len(df_active)/len(df_all)*100:>5.1f}%)")
print(f"INACTIVOS: {len(df_inactive):>8,} ({len(df_inactive)/len(df_all)*100:>5.1f}%)")
print()

# =============================================================================
# HEAD(10) - ACTIVOS
# =============================================================================
print("="*100)
print("HEAD(10) - TICKERS ACTIVOS")
print("="*100)
cols = ["ticker", "name", "market", "primary_exchange", "type", "active", "cik"]
display(df_active.select(cols).head(10))

# =============================================================================
# HEAD(10) - INACTIVOS
# =============================================================================
print()
print("="*100)
print(f"HEAD(10) - TICKERS INACTIVOS (Total: {len(df_inactive):,})")
print("="*100)
display(df_inactive.select(cols).head(10))

# =============================================================================
# TAIL(10) - INACTIVOS  
# =============================================================================
print()
print("="*100)
print("TAIL(10) - TICKERS INACTIVOS")
print("="*100)
display(df_inactive.select(cols).tail(10))

# =============================================================================
# COMPARACIÓN POR TIPO
# =============================================================================
print()
print("="*100)
print("COMPARACIÓN: DISTRIBUCIÓN POR TIPO DE ACTIVO")
print("="*100)

print("\n📊 ACTIVOS:")
type_active = (
    df_active.group_by("type")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_active) * 100).alias("percentage"))
    .sort("count", descending=True)
    .head(10)
)
display(type_active)

print("\n❌ INACTIVOS:")
type_inactive = (
    df_inactive.group_by("type")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_inactive) * 100).alias("percentage"))
    .sort("count", descending=True)
    .head(10)
)
display(type_inactive)

# =============================================================================
# COMPARACIÓN POR EXCHANGE
# =============================================================================
print()
print("="*100)
print("COMPARACIÓN: DISTRIBUCIÓN POR EXCHANGE")
print("="*100)

print("\n📊 ACTIVOS:")
exchange_active = (
    df_active.group_by("primary_exchange")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_active) * 100).alias("percentage"))
    .sort("count", descending=True)
)
display(exchange_active)

print("\n❌ INACTIVOS:")
exchange_inactive = (
    df_inactive.group_by("primary_exchange")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_inactive) * 100).alias("percentage"))
    .sort("count", descending=True)
    .head(10)
)
display(exchange_inactive)

# =============================================================================
# RESUMEN EJECUTIVO
# =============================================================================
print()
print("="*100)
print("RESUMEN EJECUTIVO")
print("="*100)
print(f"""
📊 UNIVERSO TOTAL: {len(df_all):,} tickers

ACTIVOS ({len(df_active):,}):
   • Common Stocks (CS):  {len(df_active.filter(pl.col('type') == 'CS')):>6,}
   • ETFs:                {len(df_active.filter(pl.col('type') == 'ETF')):>6,}
   • Nasdaq (XNAS):       {len(df_active.filter(pl.col('primary_exchange') == 'XNAS')):>6,}
   • NYSE (XNYS):         {len(df_active.filter(pl.col('primary_exchange') == 'XNYS')):>6,}

INACTIVOS ({len(df_inactive):,}):
   • Common Stocks (CS):  {len(df_inactive.filter(pl.col('type') == 'CS')):>6,}
   • Warrants:            {len(df_inactive.filter(pl.col('type') == 'WARRANT')):>6,}
   • ETFs:                {len(df_inactive.filter(pl.col('type') == 'ETF')):>6,}
   • Delisted/Merged:     {len(df_inactive):,} (100% inactivos)
""")

print("="*100)
print("ANÁLISIS COMPLETADO")
print("="*100)

RESUMEN GENERAL


active,count,percentage
bool,i64,f64
true,11853,34.47644
false,22527,65.52356



TOTAL TICKERS: 34,380

ACTIVOS:     11,853 ( 34.5%)
INACTIVOS:   22,527 ( 65.5%)

HEAD(10) - TICKERS ACTIVOS


ticker,name,market,primary_exchange,type,active,cik
str,str,str,str,str,bool,str
"""A""","""Agilent Technologies Inc.""","""stocks""","""XNYS""","""CS""",true,"""0001090872"""
"""AA""","""Alcoa Corporation""","""stocks""","""XNYS""","""CS""",true,"""0001675149"""
"""AAA""","""Alternative Access First Prior…","""stocks""","""ARCX""","""ETF""",true,"""0001776878"""
"""AAAA""","""Amplius Aggressive Asset Alloc…","""stocks""","""BATS""","""ETF""",true,null
"""AAAU""","""Goldman Sachs Physical Gold ET…","""stocks""","""BATS""","""ETF""",true,"""0001708646"""
"""AACB""","""Artius II Acquisition Inc. Cla…","""stocks""","""XNAS""","""CS""",true,"""0002034334"""
"""AACBR""","""Artius II Acquisition Inc. Rig…","""stocks""","""XNAS""","""RIGHT""",true,"""0002034334"""
"""AACBU""","""Artius II Acquisition Inc. Uni…","""stocks""","""XNAS""","""UNIT""",true,"""0002034334"""
"""AACG""","""ATA Creativity Global American…","""stocks""","""XNAS""","""ADRC""",true,"""0001420529"""



HEAD(10) - TICKERS INACTIVOS (Total: 22,527)


...


COMPARACIÓN: DISTRIBUCIÓN POR EXCHANGE

📊 ACTIVOS:


primary_exchange,count,percentage
str,u32,f64
"""XNAS""",5133,43.305492
"""XNYS""",2880,24.297646
"""ARCX""",2475,20.88079
"""BATS""",1063,8.968194
"""XASE""",302,2.547878



❌ INACTIVOS:


primary_exchange,count,percentage
str,u32,f64
"""XNYS""",7926,35.184445
"""XNAS""",6599,29.293736
null,4166,18.493364
"""ARCX""",1824,8.09695
"""XASE""",1658,7.360057
"""BATS""",353,1.567008
"""XBOS""",1,0.004439



RESUMEN EJECUTIVO

📊 UNIVERSO TOTAL: 34,380 tickers

ACTIVOS (11,853):
   • Common Stocks (CS):   5,229
   • ETFs:                 4,365
   • Nasdaq (XNAS):        5,133
   • NYSE (XNYS):          2,880

INACTIVOS (22,527):
   • Common Stocks (CS):   6,242
   • Warrants:             1,724
   • ETFs:                 1,363
   • Delisted/Merged:     22,527 (100% inactivos)

ANÁLISIS COMPLETADO


In [1]:
"""
Análisis: Qué nos queda al aplicar filtro CS (Common Stock)
Muestra el antes y después del filtrado
"""
import polars as pl
from pathlib import Path
import os

# Configurar
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
os.chdir(project_root)

# Cargar datos
df = pl.read_parquet("raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-24/tickers_all.parquet")

print("=" * 100)
print("QUE NOS QUEDA AL APLICAR FILTRO CS (Common Stock)")
print("=" * 100)

print(f"\n[ANTES DEL FILTRO] UNIVERSO TOTAL: {len(df):,} tickers (TODOS los tipos)")
print()

# Separar activos e inactivos ANTES del filtro
df_active_all = df.filter(pl.col("active") == True)
df_inactive_all = df.filter(pl.col("active") == False)

print("COMPOSICION ANTES DEL FILTRO:")
print(f"  ACTIVOS (TODOS los tipos):   {len(df_active_all):>7,}")
print(f"    ├─ CS (Common Stock):       {len(df_active_all.filter(pl.col('type') == 'CS')):>7,}")
print(f"    ├─ ETF:                     {len(df_active_all.filter(pl.col('type') == 'ETF')):>7,}")
print(f"    ├─ WARRANT:                 {len(df_active_all.filter(pl.col('type') == 'WARRANT')):>7,}")
print(f"    ├─ PFD (Preferred):         {len(df_active_all.filter(pl.col('type') == 'PFD')):>7,}")
print(f"    └─ Otros:                   {len(df_active_all) - len(df_active_all.filter(pl.col('type').is_in(['CS', 'ETF', 'WARRANT', 'PFD']))):>7,}")
print()
print(f"  INACTIVOS (TODOS los tipos): {len(df_inactive_all):>7,}")
print(f"    ├─ CS (Common Stock):       {len(df_inactive_all.filter(pl.col('type') == 'CS')):>7,}")
print(f"    ├─ WARRANT:                 {len(df_inactive_all.filter(pl.col('type') == 'WARRANT')):>7,}")
print(f"    ├─ ETF:                     {len(df_inactive_all.filter(pl.col('type') == 'ETF')):>7,}")
print(f"    └─ Otros:                   {len(df_inactive_all) - len(df_inactive_all.filter(pl.col('type').is_in(['CS', 'WARRANT', 'ETF']))):>7,}")

print("\n" + "=" * 100)
print("APLICANDO FILTRO: type = 'CS' (Common Stock)")
print("=" * 100)

# Filtrar solo CS
df_cs = df.filter(pl.col("type") == "CS")
df_cs_active = df_cs.filter(pl.col("active") == True)
df_cs_inactive = df_cs.filter(pl.col("active") == False)

print(f"\n[DESPUES DEL FILTRO] QUEDAN: {len(df_cs):,} tickers (SOLO Common Stock)")
print()
print("COMPOSICION DESPUES DEL FILTRO:")
print(f"  ACTIVOS CS:                   {len(df_cs_active):>7,} ({len(df_cs_active)/len(df_cs)*100:>5.1f}% de CS)")
print(f"  INACTIVOS CS:                 {len(df_cs_inactive):>7,} ({len(df_cs_inactive)/len(df_cs)*100:>5.1f}% de CS)")

print("\n" + "=" * 100)
print("APLICANDO FILTRO ADICIONAL: primary_exchange IN ('XNAS', 'XNYS')")
print("=" * 100)

# Filtrar CS + XNAS/XNYS
df_cs_exchanges = df_cs.filter(pl.col("primary_exchange").is_in(["XNAS", "XNYS"]))
df_cs_exchanges_active = df_cs_exchanges.filter(pl.col("active") == True)
df_cs_exchanges_inactive = df_cs_exchanges.filter(pl.col("active") == False)

# Por exchange
df_cs_xnas = df_cs_exchanges.filter(pl.col("primary_exchange") == "XNAS")
df_cs_xnys = df_cs_exchanges.filter(pl.col("primary_exchange") == "XNYS")

print(f"\n[DESPUES DE AMBOS FILTROS] QUEDAN: {len(df_cs_exchanges):,} tickers")
print("(Solo Common Stock en NASDAQ o NYSE)")
print()
print("COMPOSICION FINAL:")
print(f"  TOTAL CS en XNAS/XNYS:        {len(df_cs_exchanges):>7,}")
print(f"    ├─ ACTIVOS:                 {len(df_cs_exchanges_active):>7,} ({len(df_cs_exchanges_active)/len(df_cs_exchanges)*100:>5.1f}%)")
print(f"    └─ INACTIVOS:               {len(df_cs_exchanges_inactive):>7,} ({len(df_cs_exchanges_inactive)/len(df_cs_exchanges)*100:>5.1f}%)")
print()
print("  POR EXCHANGE:")
print(f"    ├─ NASDAQ (XNAS):           {len(df_cs_xnas):>7,} ({len(df_cs_xnas)/len(df_cs_exchanges)*100:>5.1f}%)")
print(f"    │  ├─ Activos:              {len(df_cs_xnas.filter(pl.col('active') == True)):>7,}")
print(f"    │  └─ Inactivos:            {len(df_cs_xnas.filter(pl.col('active') == False)):>7,}")
print(f"    └─ NYSE (XNYS):             {len(df_cs_xnys):>7,} ({len(df_cs_xnys)/len(df_cs_exchanges)*100:>5.1f}%)")
print(f"       ├─ Activos:              {len(df_cs_xnys.filter(pl.col('active') == True)):>7,}")
print(f"       └─ Inactivos:            {len(df_cs_xnys.filter(pl.col('active') == False)):>7,}")

print("\n" + "=" * 100)
print("RESUMEN: QUE SE ELIMINA CON EL FILTRO CS")
print("=" * 100)

eliminados = len(df) - len(df_cs)
print(f"\nELIMINADOS: {eliminados:,} tickers (que NO son Common Stock)")
print(f"  ├─ ETFs:                      {len(df.filter(pl.col('type') == 'ETF')):>7,}")
print(f"  ├─ Warrants:                  {len(df.filter(pl.col('type') == 'WARRANT')):>7,}")
print(f"  ├─ Preferred (PFD):           {len(df.filter(pl.col('type') == 'PFD')):>7,}")
print(f"  ├─ Funds:                     {len(df.filter(pl.col('type') == 'FUND')):>7,}")
print(f"  ├─ Units:                     {len(df.filter(pl.col('type') == 'UNIT')):>7,}")
print(f"  ├─ ADRs:                      {len(df.filter(pl.col('type') == 'ADRC')):>7,}")
print(f"  ├─ Rights:                    {len(df.filter(pl.col('type') == 'RIGHT')):>7,}")
print(f"  ├─ Indices:                   {len(df.filter(pl.col('type') == 'INDEX')):>7,}")
print(f"  ├─ Sin tipo (None):           {len(df.filter(pl.col('type').is_null())):>7,}")

otros_count = eliminados - sum([
    len(df.filter(pl.col('type') == t)) 
    for t in ['ETF', 'WARRANT', 'PFD', 'FUND', 'UNIT', 'ADRC', 'RIGHT', 'INDEX']
]) - len(df.filter(pl.col('type').is_null()))
print(f"  └─ Otros:                     {otros_count:>7,}")

print(f"\nMANTENIDOS: {len(df_cs):,} Common Stocks (CS)")
print(f"  De los cuales {len(df_cs_exchanges):,} estan en NASDAQ o NYSE")

print("\n" + "=" * 100)
print("CONCLUSION: Este es el universo base (10,599 CS) para descargar ticker details")
print("Proximo paso: Filtrar por market_cap < $2B")
print("=" * 100)

QUE NOS QUEDA AL APLICAR FILTRO CS (Common Stock)

[ANTES DEL FILTRO] UNIVERSO TOTAL: 34,380 tickers (TODOS los tipos)

COMPOSICION ANTES DEL FILTRO:
  ACTIVOS (TODOS los tipos):    11,853
    ├─ CS (Common Stock):         5,229
    ├─ ETF:                       4,365
    ├─ WARRANT:                     417
    ├─ PFD (Preferred):             441
    └─ Otros:                     1,401

  INACTIVOS (TODOS los tipos):  22,527
    ├─ CS (Common Stock):         6,242
    ├─ WARRANT:                   1,724
    ├─ ETF:                       1,363
    └─ Otros:                    13,198

APLICANDO FILTRO: type = 'CS' (Common Stock)

[DESPUES DEL FILTRO] QUEDAN: 11,471 tickers (SOLO Common Stock)

COMPOSICION DESPUES DEL FILTRO:
  ACTIVOS CS:                     5,229 ( 45.6% de CS)
  INACTIVOS CS:                   6,242 ( 54.4% de CS)

APLICANDO FILTRO ADICIONAL: primary_exchange IN ('XNAS', 'XNYS')

[DESPUES DE AMBOS FILTROS] QUEDAN: 10,599 tickers
(Solo Common Stock en NASDAQ o NYSE)

CO

In [3]:
import pandas as pd
from pathlib import Path

# Cargar con pandas (más lento pero más compatible)
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
snapshot_path = project_root / "raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-24/tickers_all.parquet"

print("Cargando datos...")
df = pd.read_parquet(snapshot_path)
print(f"✓ Datos cargados: {len(df):,} tickers\n")

# Distribución por TYPE
type_dist = df['type'].fillna('None').value_counts().reset_index()
type_dist.columns = ['Tipo', 'Cantidad']
type_dist['% Total'] = (type_dist['Cantidad'] / len(df) * 100).round(1)

# Descripciones
type_descriptions = {
    'CS': 'Common Stock (acciones comunes)',
    'ETF': 'Exchange Traded Funds',
    'PFD': 'Preferred Stock (acciones preferentes)',
    'SP': 'Security Pass-Through',
    'WARRANT': 'Warrants',
    'UNIT': 'Units',
    'ADRC': 'ADR Common',
    'FUND': 'Mutual Funds',
    'RIGHT': 'Rights',
    'INDEX': 'Indices',
    'ETN': 'Exchange Traded Notes',
    'ETS': 'Exchange Traded Shares',
    'ETV': 'Exchange Traded Vehicles',
    'ADRP': 'ADR Preferred',
    'ADRR': 'ADR Rights',
    'None': 'Sin clasificar'
}

type_dist['Descripción'] = type_dist['Tipo'].map(type_descriptions)
display(type_dist)

# Información de columnas
print(f"\nCOLUMNAS ({len(df.columns)}):")
print(df.dtypes)
print(f"\nNULLS:")
print(df.isnull().sum())

Cargando datos...
✓ Datos cargados: 34,380 tickers



,Tipo,Cantidad,% Total,Descripción
0,CS,11471,33.4,Common Stock (acciones comunes)
1,None,6830,19.9,Sin clasificar
2,ETF,5728,16.7,Exchange Traded Funds
3,PFD,2206,6.4,Preferred Stock (acciones preferentes)
4,SP,2164,6.3,Security Pass-Through
5,WARRANT,2141,6.2,Warrants
6,UNIT,1282,3.7,Units
7,ADRC,680,2.0,ADR Common
8,FUND,573,1.7,Mutual Funds
9,RIGHT,410,1.2,Rights



COLUMNAS (14):
ticker              object
name                object
market              object
locale              object
primary_exchange    object
type                object
active                bool
currency_name       object
cik                 object
composite_figi      object
share_class_figi    object
last_updated_utc    object
snapshot_date       object
delisted_utc        object
dtype: object

NULLS:
ticker                  0
name                    9
market                  0
locale                  0
primary_exchange     4166
type                 6830
active                  0
currency_name           0
cik                  5280
composite_figi      19368
share_class_figi    20115
last_updated_utc        0
snapshot_date           0
delisted_utc        12367
dtype: int64


**CODIGO PARA LANZAR LA DESCARGA DE TIKERS DE TODOS LOS CS activos e inactovos**

```sh
cd D:/04_TRADING_SMALLCAPS && POLYGON_API_KEY= 
python scripts/fase_A_universo/filter_and_enrich_universe.py 2>&1 &
```

In [5]:
"""
Verificación del Universo Híbrido (Activos < $2B + Inactivos ALL)
Análisis completo sin sesgo de supervivencia
"""
import polars as pl
from pathlib import Path
import os

# Configurar
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
os.chdir(project_root)

print("=" * 100)
print("VERIFICACIÓN DEL UNIVERSO HÍBRIDO (SIN SESGO DE SUPERVIVENCIA)")
print("=" * 100)

# =============================================================================
# PASO 1: Verificar snapshot original (TODOS los tipos)
# =============================================================================
print("\n[PASO 1] SNAPSHOT ORIGINAL (TODOS LOS TIPOS)")
print("-" * 100)

snapshot_path = "raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-24/tickers_all.parquet"
df_original = pl.read_parquet(snapshot_path)

print(f"Total tickers originales: {len(df_original):,}")
print(f"  - Activos:   {len(df_original.filter(pl.col('active') == True)):,}")
print(f"  - Inactivos: {len(df_original.filter(pl.col('active') == False)):,}")

# Distribución por tipo
print("\nDistribución por tipo (top 5):")
type_dist = df_original.group_by("type").len(name="count").sort("count", descending=True)
for row in type_dist.head(5).iter_rows(named=True):
    tipo = row['type'] if row['type'] else 'None'
    count = row['count']
    pct = count / len(df_original) * 100
    print(f"  {tipo:<15} {count:>6,} ({pct:>5.1f}%)")

# =============================================================================
# PASO 2: Verificar filtro CS + XNAS/XNYS
# =============================================================================
print("\n[PASO 2] FILTRO CS + XNAS/XNYS (BASE)")
print("-" * 100)

cs_filtered_path = "processed/universe/cs_all_xnas_xnys.parquet"
df_cs_filtered = pl.read_parquet(cs_filtered_path)

print(f"CS en XNAS/XNYS: {len(df_cs_filtered):,}")
print(f"  - Activos:   {len(df_cs_filtered.filter(pl.col('active') == True)):,}")
print(f"  - Inactivos: {len(df_cs_filtered.filter(pl.col('active') == False)):,}")
print()
print("Distribución por exchange:")
for exchange in ["XNAS", "XNYS"]:
    total = len(df_cs_filtered.filter(pl.col("primary_exchange") == exchange))
    active = len(df_cs_filtered.filter((pl.col("primary_exchange") == exchange) & (pl.col("active") == True)))
    inactive = total - active
    print(f"  {exchange}: {total:>6,} total ({active:>5,} activos, {inactive:>5,} inactivos)")

# =============================================================================
# PASO 3: Verificar ticker details descargados
# =============================================================================
print("\n[PASO 3] TICKER DETAILS DESCARGADOS")
print("-" * 100)

details_path = "raw/polygon/reference/ticker_details/ticker_details_2025-10-24.parquet"
df_details = pl.read_parquet(details_path)

print(f"Total details descargados: {len(df_details):,}")
print(f"Con market_cap data:       {len(df_details.filter(pl.col('market_cap').is_not_null())):,}")
print(f"Sin market_cap (null):     {len(df_details.filter(pl.col('market_cap').is_null())):,}")

# Análisis de activos/inactivos en details
if 'active' in df_details.columns:
    active_details = len(df_details.filter(pl.col('active') == True))
    inactive_details = len(df_details.filter(pl.col('active') == False))
    print(f"\nBreakdown por active:")
    print(f"  - Activos:   {active_details:,}")
    print(f"  - Inactivos: {inactive_details:,}")

# =============================================================================
# PASO 4: Verificar universo HÍBRIDO (activos < $2B + inactivos ALL)
# =============================================================================
print("\n[PASO 4] UNIVERSO HÍBRIDO FINAL")
print("-" * 100)

hybrid_path = "processed/universe/cs_xnas_xnys_hybrid_2025-10-24.parquet"
df_hybrid = pl.read_parquet(hybrid_path)

print(f"Total universo híbrido: {len(df_hybrid):,}")

# Verificar si existe columna 'active'
if 'active' in df_hybrid.columns:
    hybrid_active = len(df_hybrid.filter(pl.col("active") == True))
    hybrid_inactive = len(df_hybrid) - hybrid_active
    print(f"  - Activos < $2B:  {hybrid_active:>6,} ({hybrid_active/len(df_hybrid)*100:>5.1f}%)")
    print(f"  - Inactivos ALL:  {hybrid_inactive:>6,} ({hybrid_inactive/len(df_hybrid)*100:>5.1f}%)")
else:
    print("  [Nota: columna 'active' no encontrada en el resultado final]")

# Distribución por exchange
if 'primary_exchange' in df_hybrid.columns:
    print()
    print("Distribución por exchange:")
    for exchange in ["XNAS", "XNYS"]:
        total = len(df_hybrid.filter(pl.col("primary_exchange") == exchange))
        active = len(df_hybrid.filter((pl.col("primary_exchange") == exchange) & (pl.col("active") == True)))
        inactive = total - active
        pct = total / len(df_hybrid) * 100
        print(f"  {exchange}: {total:>6,} ({pct:>5.1f}%) - {active:>5,} activos, {inactive:>5,} inactivos")

# =============================================================================
# PASO 5: Análisis de market_cap (solo en activos)
# =============================================================================
print("\n[PASO 5] ANÁLISIS DE MARKET CAP (SOLO ACTIVOS)")
print("-" * 100)

# Verificar cuántos activos tienen market_cap en el híbrido
if 'active' in df_hybrid.columns:
    activos_hybrid = df_hybrid.filter(pl.col('active') == True)
    
    # Hacer join con details para obtener market_cap
    df_with_details = activos_hybrid.join(
        df_details.select(['ticker', 'market_cap']),
        on='ticker',
        how='left'
    )
    
    with_cap = df_with_details.filter(pl.col('market_cap').is_not_null())
    without_cap = df_with_details.filter(pl.col('market_cap').is_null())
    
    print(f"Activos en híbrido:         {len(activos_hybrid):,}")
    print(f"  - Con market_cap:         {len(with_cap):,}")
    print(f"  - Sin market_cap:         {len(without_cap):,}")
    
    if len(with_cap) > 0:
        print()
        print("Estadísticas de market cap (activos):")
        stats = with_cap.select(pl.col("market_cap")).describe()
        display(stats)
        
        print()
        print("Rangos de market cap (activos):")
        ranges = [
            ("Nano cap (< $50M)", 0, 50_000_000),
            ("Micro cap ($50M - $300M)", 50_000_000, 300_000_000),
            ("Small cap ($300M - $2B)", 300_000_000, 2_000_000_000),
        ]
        
        for label, min_val, max_val in ranges:
            count = len(with_cap.filter(
                (pl.col("market_cap") >= min_val) & (pl.col("market_cap") < max_val)
            ))
            pct = count / len(with_cap) * 100
            print(f"  {label}: {count:>6,} ({pct:>5.1f}%)")
        
        # Verificar si hay alguno >= $2B (error en filtrado)
        over_2b = len(with_cap.filter(pl.col("market_cap") >= 2_000_000_000))
        if over_2b > 0:
            print(f"\n⚠️  ADVERTENCIA: {over_2b} activos con market_cap >= $2B (error en filtrado)")

print("\nNOTA: Los 5,594 inactivos NO tienen market_cap (Polygon no proporciona)")

# =============================================================================
# PASO 6: Muestreo de datos
# =============================================================================
print("\n[PASO 6] MUESTREO DE DATOS FINALES")
print("-" * 100)

print("\nPrimeros 10 tickers ACTIVOS (ordenados alfabéticamente):")
cols_display = ["ticker", "name", "primary_exchange", "active"]

if 'ticker' in df_hybrid.columns and 'active' in df_hybrid.columns:
    activos_sample = df_hybrid.filter(pl.col('active') == True).sort("ticker")
    display(activos_sample.select([col for col in cols_display if col in activos_sample.columns]).head(10))

print("\nPrimeros 10 tickers INACTIVOS (ordenados alfabéticamente):")
if 'ticker' in df_hybrid.columns and 'active' in df_hybrid.columns:
    inactivos_sample = df_hybrid.filter(pl.col('active') == False).sort("ticker")
    display(inactivos_sample.select([col for col in cols_display if col in inactivos_sample.columns]).head(10))

# =============================================================================
# PASO 7: Resumen ejecutivo
# =============================================================================
print("\n" + "=" * 100)
print("RESUMEN EJECUTIVO")
print("=" * 100)

print(f"""
PIPELINE COMPLETO:
  1. Snapshot original (todos tipos): {len(df_original):>6,} tickers
  2. Filtro CS:                       {len(df_original.filter(pl.col('type') == 'CS')):>6,} tickers
  3. Filtro CS + XNAS/XNYS:           {len(df_cs_filtered):>6,} tickers
  4. Details descargados:             {len(df_details):>6,} tickers
  5. Con market_cap data:             {len(df_details.filter(pl.col('market_cap').is_not_null())):>6,} tickers (solo activos)
  6. Universo HÍBRIDO final:          {len(df_hybrid):>6,} tickers ✅

COMPOSICIÓN DEL UNIVERSO HÍBRIDO:
  - Activos < $2B:    {hybrid_active:>6,} tickers ({hybrid_active/len(df_hybrid)*100:>5.1f}%) [filtrados por market_cap]
  - Inactivos ALL:    {hybrid_inactive:>6,} tickers ({hybrid_inactive/len(df_hybrid)*100:>5.1f}%) [TODOS incluidos]
  
  TOTAL:              {len(df_hybrid):>6,} tickers

DISTRIBUCIÓN:
  - NASDAQ (XNAS):    {len(df_hybrid.filter(pl.col('primary_exchange') == 'XNAS')):>6,} tickers
  - NYSE (XNYS):      {len(df_hybrid.filter(pl.col('primary_exchange') == 'XNYS')):>6,} tickers

CARACTERÍSTICAS:
  ✅ Common Stock (CS) solamente
  ✅ NASDAQ (XNAS) y NYSE (XNYS) únicamente
  ✅ Activos filtrados por market_cap < $2B
  ✅ Inactivos incluidos SIN FILTRAR (evita survivorship bias)
  ✅ Sin sesgo de supervivencia

FUNDAMENTO (López de Prado):
  "Survivorship bias is one of the most severe biases in backtesting"
  - Los inactivos son CRÍTICOS para entrenar modelos de pump & dump
  - Contienen las señales más fuertes de pumps terminales
  - Permiten feature engineering: days_to_delisting, is_terminal_pump

ARCHIVOS GENERADOS:
  ✅ {cs_filtered_path}
  ✅ {details_path}
  ✅ {hybrid_path}
  ✅ processed/universe/cs_xnas_xnys_hybrid_2025-10-24.csv
""")

print("=" * 100)
print("VERIFICACIÓN COMPLETADA")
print("=" * 100)

VERIFICACIÓN DEL UNIVERSO HÍBRIDO (SIN SESGO DE SUPERVIVENCIA)

[PASO 1] SNAPSHOT ORIGINAL (TODOS LOS TIPOS)
----------------------------------------------------------------------------------------------------
Total tickers originales: 34,380
  - Activos:   11,853
  - Inactivos: 22,527

Distribución por tipo (top 5):
  CS              11,471 ( 33.4%)
  None             6,830 ( 19.9%)
  ETF              5,728 ( 16.7%)
  PFD              2,206 (  6.4%)
  SP               2,164 (  6.3%)

[PASO 2] FILTRO CS + XNAS/XNYS (BASE)
----------------------------------------------------------------------------------------------------
CS en XNAS/XNYS: 10,599
  - Activos:   5,005
  - Inactivos: 5,594

Distribución por exchange:
  XNAS:  6,227 total (3,268 activos, 2,959 inactivos)
  XNYS:  4,372 total (1,737 activos, 2,635 inactivos)

[PASO 3] TICKER DETAILS DESCARGADOS
----------------------------------------------------------------------------------------------------
Total details descargados: 10,5

statistic,market_cap
str,f64
"""count""",3092.0
"""null_count""",0.0
"""mean""",4.4097e8
"""std""",5.1326e8
"""min""",765483.5278
"""25%""",5.1859e7
"""50%""",2.2937e8
"""75%""",6.6841e8
"""max""",2.0000e9



Rangos de market cap (activos):
  Nano cap (< $50M):    759 ( 24.5%)
  Micro cap ($50M - $300M):    989 ( 32.0%)
  Small cap ($300M - $2B):  1,344 ( 43.5%)

NOTA: Los 5,594 inactivos NO tienen market_cap (Polygon no proporciona)

[PASO 6] MUESTREO DE DATOS FINALES
----------------------------------------------------------------------------------------------------

Primeros 10 tickers ACTIVOS (ordenados alfabéticamente):


ticker,name,primary_exchange,active
str,str,str,bool
"""AACB""","""Artius II Acquisition Inc. Cla…","""XNAS""",true
"""AACI""","""Armada Acquisition Corp. II Cl…","""XNAS""",true
"""AAM""","""AA Mission Acquisition Corp.""","""XNYS""",true
"""AAME""","""Atlantic American Corp""","""XNAS""",true
"""AAMI""","""Acadian Asset Management Inc.""","""XNYS""",true
"""AARD""","""Aardvark Therapeutics, Inc. Co…","""XNAS""",true
"""AAT""","""AMERICAN ASSETS TRUST, INC.""","""XNYS""",true
"""AAUC""","""Allied Gold Corporation""","""XNYS""",true
"""ABAT""","""American Battery Technology Co…","""XNAS""",true



Primeros 10 tickers INACTIVOS (ordenados alfabéticamente):


ticker,name,primary_exchange,active
str,str,str,bool
"""AABA""","""Altaba Inc. Common Stock""","""XNAS""",false
"""AAC""","""Ares Acquisition Corporation""","""XNYS""",false
"""AACQ""","""Artius Acquisition Inc. Class …","""XNAS""",false
"""AACT""","""Ares Acquisition Corporation I…","""XNYS""",false
"""AADI""","""Aadi Bioscience, Inc. Common S…","""XNAS""",false
"""AAGR""","""African Agriculture Holdings I…","""XNAS""",false
"""AAI""","""AIRTRAN HOLDINGS INC""","""XNYS""",false
"""AAIC""","""Arlington Asset Investment Cor…","""XNYS""",false
"""AAN""","""The Aaron's Company, Inc.""","""XNYS""",false



RESUMEN EJECUTIVO

PIPELINE COMPLETO:
  1. Snapshot original (todos tipos): 34,380 tickers
  2. Filtro CS:                       11,471 tickers
  3. Filtro CS + XNAS/XNYS:           10,599 tickers
  4. Details descargados:             10,592 tickers
  5. Con market_cap data:              4,884 tickers (solo activos)
  6. Universo HÍBRIDO final:           8,686 tickers ✅

COMPOSICIÓN DEL UNIVERSO HÍBRIDO:
  - Activos < $2B:     3,092 tickers ( 35.6%) [filtrados por market_cap]
  - Inactivos ALL:     5,594 tickers ( 64.4%) [TODOS incluidos]

  TOTAL:               8,686 tickers

DISTRIBUCIÓN:
  - NASDAQ (XNAS):     5,414 tickers
  - NYSE (XNYS):       3,272 tickers

CARACTERÍSTICAS:
  ✅ Common Stock (CS) solamente
  ✅ NASDAQ (XNAS) y NYSE (XNYS) únicamente
  ✅ Activos filtrados por market_cap < $2B
  ✅ Inactivos incluidos SIN FILTRAR (evita survivorship bias)
  ✅ Sin sesgo de supervivencia

FUNDAMENTO (López de Prado):
  "Survivorship bias is one of the most severe biases in backtesting

In [10]:
# Análisis de Ticker Details descargados desde Polygon API
import polars as pl
import pandas as pd
from pathlib import Path

# Cargar ticker details
df_details = pl.read_parquet('raw/polygon/reference/ticker_details/ticker_details_2025-10-24.parquet')

print(f"Total tickers con detalles descargados: {len(df_details):,}")
print(f"\nColumnas disponibles ({len(df_details.columns)}): {df_details.columns}")

print("\n" + "="*80)
print("ESTADÍSTICAS DE COMPLETITUD POR COLUMNA")
print("="*80)

# Crear tabla de estadísticas
stats = []
for col in df_details.columns:
    non_null = len(df_details[col].drop_nulls())
    pct = (non_null/len(df_details))*100
    stats.append({
        'Columna': col,
        'No-Null': non_null,
        'Total': len(df_details),
        'Completitud': f'{pct:.1f}%'
    })

df_stats = pd.DataFrame(stats)
print(df_stats.to_string(index=False))

print("\n" + "="*80)
print("HALLAZGO CRÍTICO")
print("="*80)
print(f"✅ Tickers CON market_cap: {len(df_details.filter(pl.col('market_cap').is_not_null())):,} (todos activos)")
print(f"❌ Tickers SIN market_cap: {len(df_details.filter(pl.col('market_cap').is_null())):,} (todos inactivos)")

print("\n" + "="*80)
print("EJEMPLO: TICKER ACTIVO (con market_cap)")
print("="*80)
activo = df_details.filter(pl.col('market_cap').is_not_null()).head(1)
activo_pd = activo.select(['ticker', 'name', 'active', 'market_cap', 
                           'share_class_shares_outstanding', 'total_employees', 
                           'sic_description', 'list_date', 'homepage_url']).to_pandas()
print(activo_pd.to_string(index=False))

print("\n" + "="*80)
print("EJEMPLO: TICKER INACTIVO (sin market_cap)")
print("="*80)
inactivo = df_details.filter(pl.col('market_cap').is_null()).head(1)
inactivo_pd = inactivo.select(['ticker', 'name', 'active', 'market_cap', 
                               'share_class_shares_outstanding', 'sic_description', 
                               'list_date']).to_pandas()

inactivo_pd

Total tickers con detalles descargados: 10,592

Columnas disponibles (28): ['ticker', 'error', 'name', 'market', 'locale', 'primary_exchange', 'type', 'active', 'currency_name', 'cik', 'market_cap', 'phone_number', 'address', 'description', 'sic_code', 'sic_description', 'ticker_root', 'homepage_url', 'list_date', 'share_class_shares_outstanding', 'weighted_shares_outstanding', 'round_lot', 'fetch_date', 'composite_figi', 'share_class_figi', 'total_employees', 'branding', 'ticker_suffix']

ESTADÍSTICAS DE COMPLETITUD POR COLUMNA
                       Columna  No-Null  Total Completitud
                        ticker    10592  10592      100.0%
                         error     5358  10592       50.6%
                          name     5234  10592       49.4%
                        market     5234  10592       49.4%
                        locale     5234  10592       49.4%
              primary_exchange     5234  10592       49.4%
                          type     5234  10592      

,ticker,name,active,market_cap,share_class_shares_outstanding,sic_description,list_date
0,AAPC,None,None,NaN,NaN,None,None


In [12]:
# Análisis COMPLETO de Ticker Details - Formato Transpuesto
import polars as pl
import pandas as pd

# Cargar ticker details
df_details = pl.read_parquet('raw/polygon/reference/ticker_details/ticker_details_2025-10-24.parquet')

print("="*80)
print("EJEMPLO COMPLETO: TICKER ACTIVO (con market_cap)")
print("="*80)
activo = df_details.filter(pl.col('market_cap').is_not_null()).head(1).to_pandas()
activo_T = activo.T
activo_T.columns = ['Valor']
activo_T

EJEMPLO COMPLETO: TICKER ACTIVO (con market_cap)


,Valor
ticker,AAM
error,None
name,AA Mission Acquisition Corp.
market,stocks
locale,us
primary_exchange,XNYS
type,CS
active,True
currency_name,usd
cik,0002012964


In [18]:
print("\n" + "="*80)
print("EJEMPLO COMPLETO: TICKER INACTIVO (sin market_cap)")
print("="*80)
inactivo = df_details.filter(pl.col('market_cap').is_null()).head(1).to_pandas()
inactivo_T = inactivo.T
inactivo_T.columns = ['Valor']
inactivo_T


EJEMPLO COMPLETO: TICKER INACTIVO (sin market_cap)


,Valor
ticker,AAPC
error,not_found
name,None
market,None
locale,None
primary_exchange,None
type,None
active,None
currency_name,None
cik,None


In [16]:
print("\n" + "="*80)
print("COMPARACIÓN LADO A LADO")
print("="*80)
# Unir ambos para comparar
comparacion = pd.DataFrame({
    'Columna': activo.columns,
    'ACTIVO (ejemplo)': activo.iloc[0].values,
    'INACTIVO (ejemplo)': inactivo.iloc[0].values
})
print(comparacion.to_string(index=False))


COMPARACIÓN LADO A LADO
                       Columna                                                                                                                      ACTIVO (ejemplo) INACTIVO (ejemplo)
                        ticker                                                                                                                                   AAM               AAPC
                         error                                                                                                                                  None          not_found
                          name                                                                                                          AA Mission Acquisition Corp.               None
                        market                                                                                                                                stocks               None
                        locale                         

In [19]:
# =============================================================================
# VERIFICACIÓN DEL UNIVERSO HÍBRIDO ENRIQUECIDO
# =============================================================================

import polars as pl
import pandas as pd
from pathlib import Path

# Cargar universo enriquecido
df = pl.read_parquet('processed/universe/cs_xnas_xnys_hybrid_enriched_2025-10-24.parquet')

print("="*80)
print("UNIVERSO HÍBRIDO ENRIQUECIDO - RESUMEN")
print("="*80)
print(f"Total tickers: {len(df):,}")
print(f"Total columnas: {len(df.columns)}")
print(f"\nColumnas: {df.columns}")

# =============================================================================
# DISTRIBUCIÓN ACTIVOS vs INACTIVOS
# =============================================================================

print("\n" + "="*80)
print("DISTRIBUCIÓN: ACTIVOS vs INACTIVOS")
print("="*80)

activos = df.filter(pl.col('active') == True)
inactivos = df.filter(pl.col('active') == False)

print(f"Activos:   {len(activos):,} ({len(activos)/len(df)*100:.1f}%)")
print(f"Inactivos: {len(inactivos):,} ({len(inactivos)/len(df)*100:.1f}%)")

# =============================================================================
# COMPLETITUD DE DATOS POR SEGMENTO
# =============================================================================

print("\n" + "="*80)
print("COMPLETITUD DE DATOS CLAVE")
print("="*80)

fields_check = [
    'market_cap', 'delisted_utc', 'sic_description', 'total_employees', 
    'composite_figi', 'description', 'homepage_url', 'list_date'
]

completitud = []
for field in fields_check:
    if field in df.columns:
        # Total
        total_not_null = len(df.filter(pl.col(field).is_not_null()))
        total_pct = (total_not_null / len(df)) * 100
        
        # Activos
        activos_not_null = len(activos.filter(pl.col(field).is_not_null()))
        activos_pct = (activos_not_null / len(activos)) * 100 if len(activos) > 0 else 0
        
        # Inactivos
        inactivos_not_null = len(inactivos.filter(pl.col(field).is_not_null()))
        inactivos_pct = (inactivos_not_null / len(inactivos)) * 100 if len(inactivos) > 0 else 0
        
        completitud.append({
            'Campo': field,
            'Total': f'{total_not_null:,} ({total_pct:.1f}%)',
            'Activos': f'{activos_not_null:,} ({activos_pct:.1f}%)',
            'Inactivos': f'{inactivos_not_null:,} ({inactivos_pct:.1f}%)'
        })

df_completitud = pd.DataFrame(completitud)
print(df_completitud.to_string(index=False))

# =============================================================================
# EJEMPLO TRANSPUESTO: ACTIVO vs INACTIVO
# =============================================================================

print("\n" + "="*80)
print("COMPARACIÓN: TICKER ACTIVO vs INACTIVO (TRANSPUESTO)")
print("="*80)

# Ticker activo con market_cap
activo_sample = activos.filter(pl.col('market_cap').is_not_null()).head(1).to_pandas()
# Ticker inactivo con delisted_utc
inactivo_sample = inactivos.filter(pl.col('delisted_utc').is_not_null()).head(1).to_pandas()

# Crear comparación transpuesta
comparacion = pd.DataFrame({
    'Campo': activo_sample.columns,
    'ACTIVO (ejemplo)': activo_sample.iloc[0].values,
    'INACTIVO (ejemplo)': inactivo_sample.iloc[0].values
})

print(comparacion.to_string(index=False))

# =============================================================================
# DISTRIBUCIÓN POR EXCHANGE
# =============================================================================

print("\n" + "="*80)
print("DISTRIBUCIÓN POR EXCHANGE")
print("="*80)

exchange_dist = df.group_by('primary_exchange').agg([
    pl.count('ticker').alias('Total'),
    pl.col('active').sum().alias('Activos'),
    (pl.col('active') == False).sum().alias('Inactivos')
]).sort('Total', descending=True)

print(exchange_dist.to_pandas().to_string(index=False))

# =============================================================================
# MARKET CAP DISTRIBUTION (solo activos)
# =============================================================================

print("\n" + "="*80)
print("DISTRIBUCIÓN MARKET CAP (ACTIVOS)")
print("="*80)

activos_with_mcap = activos.filter(pl.col('market_cap').is_not_null())

mcap_stats = activos_with_mcap.select([
    pl.col('market_cap').min().alias('Min'),
    pl.col('market_cap').quantile(0.25).alias('Q1'),
    pl.col('market_cap').median().alias('Mediana'),
    pl.col('market_cap').quantile(0.75).alias('Q3'),
    pl.col('market_cap').max().alias('Max'),
    pl.col('market_cap').mean().alias('Media')
])

mcap_stats_pd = mcap_stats.to_pandas()
# Formatear en millones
for col in mcap_stats_pd.columns:
    mcap_stats_pd[col] = mcap_stats_pd[col].apply(lambda x: f'${x/1e6:.2f}M' if pd.notna(x) else 'N/A')

print(mcap_stats_pd.to_string(index=False))

# Verificar que todos los activos tengan market_cap < $2B
activos_over_2b = len(activos_with_mcap.filter(pl.col('market_cap') > 2e9))
print(f"\n✅ Activos con market_cap > $2B: {activos_over_2b} (debe ser 0)")
print(f"✅ Activos con market_cap ≤ $2B: {len(activos_with_mcap):,}")

# =============================================================================
# DELISTED DATES DISTRIBUTION (solo inactivos)
# =============================================================================

print("\n" + "="*80)
print("DISTRIBUCIÓN DELISTED_UTC (INACTIVOS)")
print("="*80)

inactivos_with_delisted = inactivos.filter(pl.col('delisted_utc').is_not_null())

print(f"Inactivos con delisted_utc: {len(inactivos_with_delisted):,} / {len(inactivos):,} ({len(inactivos_with_delisted)/len(inactivos)*100:.1f}%)")

# Extraer año de delisting
inactivos_years = inactivos_with_delisted.with_columns(
    pl.col('delisted_utc').str.slice(0, 4).alias('delisted_year')
).group_by('delisted_year').agg(
    pl.count('ticker').alias('Count')
).sort('delisted_year', descending=True).head(10)

print("\nTop 10 años con más delistings:")
print(inactivos_years.to_pandas().to_string(index=False))

# =============================================================================
# VERIFICACIÓN FINAL
# =============================================================================

print("\n" + "="*80)
print("VERIFICACIÓN FINAL")
print("="*80)

checks = [
    ("Total tickers", len(df) == 8686),
    ("Total activos", len(activos) == 3092),
    ("Total inactivos", len(inactivos) == 5594),
    ("Todos activos tienen market_cap", len(activos.filter(pl.col('market_cap').is_not_null())) == len(activos)),
    ("Ningún activo > $2B", len(activos.filter(pl.col('market_cap') > 2e9)) == 0),
    ("96%+ inactivos tienen delisted_utc", len(inactivos.filter(pl.col('delisted_utc').is_not_null())) / len(inactivos) > 0.96)
]

for check_name, result in checks:
    status = "✅ PASS" if result else "❌ FAIL"
    print(f"{status}: {check_name}")

print("\n" + "="*80)
print(f"Archivo: processed/universe/cs_xnas_xnys_hybrid_enriched_2025-10-24.parquet")
print(f"Tamaño: {Path('processed/universe/cs_xnas_xnys_hybrid_enriched_2025-10-24.parquet').stat().st_size / 1024 / 1024:.2f} MB")
print("="*80)

UNIVERSO HÍBRIDO ENRIQUECIDO - RESUMEN
Total tickers: 8,686
Total columnas: 23

Columnas: ['active', 'cik', 'composite_figi', 'currency_name', 'delisted_utc', 'description', 'homepage_url', 'last_updated_utc', 'list_date', 'locale', 'market', 'market_cap', 'name', 'primary_exchange', 'share_class_figi', 'share_class_shares_outstanding', 'sic_code', 'sic_description', 'snapshot_date', 'ticker', 'total_employees', 'type', 'weighted_shares_outstanding']

DISTRIBUCIÓN: ACTIVOS vs INACTIVOS
Activos:   3,092 (35.6%)
Inactivos: 5,594 (64.4%)

COMPLETITUD DE DATOS CLAVE
          Campo         Total        Activos     Inactivos
     market_cap 3,092 (35.6%) 3,092 (100.0%)      0 (0.0%)
   delisted_utc 5,393 (62.1%)       0 (0.0%) 5,393 (96.4%)
sic_description 2,454 (28.3%)  2,454 (79.4%)      0 (0.0%)
total_employees 2,738 (31.5%)  2,738 (88.6%)      0 (0.0%)
 composite_figi 4,812 (55.4%)  2,409 (77.9%) 2,403 (43.0%)
    description 3,092 (35.6%) 3,092 (100.0%)      0 (0.0%)
   homepage_url 2,

In [20]:
import polars as pl
import pandas as pd

# Cargar universo enriquecido
df = pl.read_parquet('processed/universe/cs_xnas_xnys_hybrid_enriched_2025-10-24.parquet')

# Separar activos e inactivos
activos = df.filter(pl.col('active') == True)
inactivos = df.filter(pl.col('active') == False)

print("="*80)
print("ANÁLISIS DE COLUMNAS: ACTIVOS vs INACTIVOS")
print("="*80)

# Analizar cada columna
analisis = []
for col in df.columns:
    # Activos
    activos_not_null = len(activos.filter(pl.col(col).is_not_null()))
    activos_pct = (activos_not_null / len(activos)) * 100 if len(activos) > 0 else 0
    
    # Inactivos
    inactivos_not_null = len(inactivos.filter(pl.col(col).is_not_null()))
    inactivos_pct = (inactivos_not_null / len(inactivos)) * 100 if len(inactivos) > 0 else 0
    
    analisis.append({
        'Columna': col,
        'Activos_Count': activos_not_null,
        'Activos_%': f'{activos_pct:.1f}%',
        'Inactivos_Count': inactivos_not_null,
        'Inactivos_%': f'{inactivos_pct:.1f}%',
        'Tiene_Activos': '✅' if activos_pct > 0 else '❌',
        'Tiene_Inactivos': '✅' if inactivos_pct > 0 else '❌'
    })

df_analisis = pd.DataFrame(analisis)
print(df_analisis.to_string(index=False))

# Resumen
print("\n" + "="*80)
print("RESUMEN")
print("="*80)

solo_activos = df_analisis[(df_analisis['Tiene_Activos'] == '✅') & (df_analisis['Tiene_Inactivos'] == '❌')]
solo_inactivos = df_analisis[(df_analisis['Tiene_Activos'] == '❌') & (df_analisis['Tiene_Inactivos'] == '✅')]
ambos = df_analisis[(df_analisis['Tiene_Activos'] == '✅') & (df_analisis['Tiene_Inactivos'] == '✅')]
ninguno = df_analisis[(df_analisis['Tiene_Activos'] == '❌') & (df_analisis['Tiene_Inactivos'] == '❌')]

print(f"\nColumnas SOLO con datos en ACTIVOS ({len(solo_activos)}):")
if len(solo_activos) > 0:
    print(solo_activos[['Columna', 'Activos_%']].to_string(index=False))
else:
    print("  (ninguna)")

print(f"\nColumnas SOLO con datos en INACTIVOS ({len(solo_inactivos)}):")
if len(solo_inactivos) > 0:
    print(solo_inactivos[['Columna', 'Inactivos_%']].to_string(index=False))
else:
    print("  (ninguna)")

print(f"\nColumnas con datos en AMBOS ({len(ambos)}):")
if len(ambos) > 0:
    print(ambos[['Columna', 'Activos_%', 'Inactivos_%']].to_string(index=False))
else:
    print("  (ninguna)")

print(f"\nColumnas SIN datos en NINGUNO ({len(ninguno)}):")
if len(ninguno) > 0:
    print(ninguno[['Columna']].to_string(index=False))
else:
    print("  (ninguna)")

ANÁLISIS DE COLUMNAS: ACTIVOS vs INACTIVOS
                       Columna  Activos_Count Activos_%  Inactivos_Count Inactivos_% Tiene_Activos Tiene_Inactivos
                        active           3092    100.0%             5594      100.0%             ✅               ✅
                           cik           3089     99.9%             5339       95.4%             ✅               ✅
                composite_figi           2409     77.9%             2403       43.0%             ✅               ✅
                 currency_name           3092    100.0%             5594      100.0%             ✅               ✅
                  delisted_utc              0      0.0%             5393       96.4%             ❌               ✅
                   description           3092    100.0%                0        0.0%             ✅               ❌
                  homepage_url           2967     96.0%                0        0.0%             ✅               ❌
              last_updated_utc       

### split & dividens

In [ ]:
# =============================================================================
# VERIFICACIÓN DE CORPORATE ACTIONS - 8,686 TICKERS
# =============================================================================

import polars as pl
import pandas as pd
from pathlib import Path

# =============================================================================
# CARGAR DATOS
# =============================================================================

print("="*80)
print("VERIFICACIÓN: SPLITS & DIVIDENDS DEL UNIVERSO")
print("="*80)

# Cargar universo
universo = pl.read_parquet('processed/universe/cs_xnas_xnys_hybrid_enriched_2025-10-24.parquet')
print(f"\nUniverso total: {len(universo):,} tickers")

# Cargar splits filtrados
splits = pl.read_parquet('processed/corporate_actions/splits_universe_2025-10-24.parquet')
print(f"Splits: {len(splits):,} registros")

# Cargar dividends filtrados
dividends = pl.read_parquet('processed/corporate_actions/dividends_universe_2025-10-24.parquet')
print(f"Dividends: {len(dividends):,} registros")

# Cargar lookup table
lookup = pl.read_parquet('processed/corporate_actions/corporate_actions_lookup_2025-10-24.parquet')
print(f"Lookup table: {len(lookup):,} tickers")

# =============================================================================
# ESTADÍSTICAS GENERALES
# =============================================================================

print("\n" + "="*80)
print("ESTADÍSTICAS GENERALES")
print("="*80)

tickers_con_splits = splits['ticker'].n_unique()
tickers_con_dividends = dividends['ticker'].n_unique()

print(f"\nTickers con splits: {tickers_con_splits:,} ({tickers_con_splits/len(universo)*100:.1f}%)")
print(f"Tickers con dividends: {tickers_con_dividends:,} ({tickers_con_dividends/len(universo)*100:.1f}%)")
print(f"Tickers sin splits: {len(universo) - tickers_con_splits:,}")
print(f"Tickers sin dividends: {len(universo) - tickers_con_dividends:,}")

# =============================================================================
# EXPLORAR SPLITS
# =============================================================================

print("\n" + "="*80)
print("ANÁLISIS DE SPLITS")
print("="*80)

print(f"\nColumnas: {splits.columns}")

# Muestra de splits
print("\nMuestra de 10 splits:")
splits_sample = splits.select(['ticker', 'execution_date', 'split_from', 'split_to', 'ratio']).head(10).to_pandas()
print(splits_sample.to_string(index=False))

# Top tickers con más splits
top_splits = (splits
    .group_by('ticker')
    .agg(pl.count('ticker').alias('count'))
    .sort('count', descending=True)
    .head(10)
    .to_pandas()
)
print("\nTop 10 tickers con más splits:")
print(top_splits.to_string(index=False))

# Reverse splits (dilución extrema)
if 'ratio' in splits.columns:
    reverse_splits = splits.filter(pl.col('ratio') < 1.0)
    print(f"\nReverse splits (ratio < 1.0): {len(reverse_splits):,} ({len(reverse_splits)/len(splits)*100:.1f}%)")
    
    # Tickers con múltiples reverse splits
    multi_reverse = (reverse_splits
        .group_by('ticker')
        .agg(pl.count('ticker').alias('reverse_count'))
        .filter(pl.col('reverse_count') >= 5)
        .sort('reverse_count', descending=True)
        .to_pandas()
    )
    print(f"Tickers con 5+ reverse splits (dilución extrema): {len(multi_reverse)}")
    if len(multi_reverse) > 0:
        print(multi_reverse.head(10).to_string(index=False))

# =============================================================================
# EXPLORAR DIVIDENDS
# =============================================================================

print("\n" + "="*80)
print("ANÁLISIS DE DIVIDENDS")
print("="*80)

print(f"\nColumnas: {dividends.columns}")

# Muestra de dividends
print("\nMuestra de 10 dividends:")
dividends_sample = dividends.select(['ticker', 'ex_dividend_date', 'cash_amount']).head(10).to_pandas()
print(dividends_sample.to_string(index=False))

# Top tickers con más dividends
top_dividends = (dividends
    .group_by('ticker')
    .agg(pl.count('ticker').alias('count'))
    .sort('count', descending=True)
    .head(10)
    .to_pandas()
)
print("\nTop 10 tickers con más dividends:")
print(top_dividends.to_string(index=False))

# Distribución por año
if 'ex_dividend_date' in dividends.columns:
    dividends_por_año = (dividends
        .with_columns(
            pl.col('ex_dividend_date').str.slice(0, 4).alias('year')
        )
        .group_by('year')
        .agg(pl.count('ticker').alias('count'))
        .sort('year')
        .to_pandas()
    )
    print("\nDividends por año:")
    print(dividends_por_año.to_string(index=False))

# =============================================================================
# LOOKUP TABLE - RESUMEN RÁPIDO
# =============================================================================

print("\n" + "="*80)
print("LOOKUP TABLE - RESUMEN POR TICKER")
print("="*80)

print(f"\nColumnas: {lookup.columns}")

# Estadísticas de la lookup
stats_lookup = lookup.select([
    pl.col('has_splits').sum().alias('con_splits'),
    pl.col('has_dividends').sum().alias('con_dividends'),
    pl.col('splits_count').sum().alias('total_splits'),
    pl.col('dividends_count').sum().alias('total_dividends')
]).to_pandas()

print("\nEstadísticas desde lookup:")
print(stats_lookup.to_string(index=False))

# Tickers con ambos (splits Y dividends)
con_ambos = lookup.filter(
    (pl.col('has_splits') == True) & 
    (pl.col('has_dividends') == True)
)
print(f"\nTickers con SPLITS Y DIVIDENDS: {len(con_ambos):,}")

# Tickers sin ninguno
sin_ninguno = lookup.filter(
    (pl.col('has_splits') == False) & 
    (pl.col('has_dividends') == False)
)
print(f"Tickers SIN corporate actions: {len(sin_ninguno):,}")

# Muestra de lookup table
print("\nMuestra de lookup table (10 tickers con más eventos):")
lookup_top = (lookup
    .with_columns(
        (pl.col('splits_count') + pl.col('dividends_count')).alias('total_events')
    )
    .sort('total_events', descending=True)
    .select(['ticker', 'name', 'active', 'has_splits', 'has_dividends', 'splits_count', 'dividends_count', 'total_events'])
    .head(10)
    .to_pandas()
)
print(lookup_top.to_string(index=False))

# =============================================================================
# BUSCAR UN TICKER ESPECÍFICO
# =============================================================================

print("\n" + "="*80)
print("BÚSQUEDA POR TICKER")
print("="*80)

# Ejemplo: Buscar AAPL (si existe en nuestro universo)
ticker_buscar = 'GOOD'  # Cambiar por el ticker que quieras

ticker_info = lookup.filter(pl.col('ticker') == ticker_buscar)
if len(ticker_info) > 0:
    print(f"\nInformación de {ticker_buscar}:")
    print(ticker_info.to_pandas().T)
    
    # Splits del ticker
    ticker_splits = splits.filter(pl.col('ticker') == ticker_buscar)
    if len(ticker_splits) > 0:
        print(f"\nSplits de {ticker_buscar}:")
        print(ticker_splits.to_pandas().to_string(index=False))
    
    # Dividends del ticker
    ticker_dividends = dividends.filter(pl.col('ticker') == ticker_buscar)
    if len(ticker_dividends) > 0:
        print(f"\nDividends de {ticker_buscar} (últimos 10):")
        print(ticker_dividends.tail(10).to_pandas().to_string(index=False))
else:
    print(f"\n{ticker_buscar} no está en nuestro universo")

print("\n" + "="*80)

VERIFICACIÓN: SPLITS & DIVIDENDS DEL UNIVERSO

Universo total: 8,686 tickers
Splits: 4,012 registros
Dividends: 94,546 registros
Lookup table: 8,686 tickers

ESTADÍSTICAS GENERALES

Tickers con splits: 2,420 (27.9%)
Tickers con dividends: 2,723 (31.3%)
Tickers sin splits: 6,266
Tickers sin dividends: 5,963

ANÁLISIS DE SPLITS

Columnas: ['execution_date', 'id', 'split_from', 'split_to', 'ticker', 'ratio']

Muestra de 10 splits:
ticker execution_date  split_from  split_to     ratio
   IOM     2001-10-01         5.0       1.0  5.000000
   MNC     2001-09-10         2.0       3.0  0.666667
  SPPI     2002-09-06        25.0       1.0 25.000000
   WRI     2002-04-16         2.0       5.0  0.400000
    DF     2002-04-24         1.0       2.0  0.500000
  EBIX     2002-10-01         8.0       1.0  8.000000
   CKC     2002-06-28         2.5       1.0  2.500000
  SQBG     2002-07-17         7.0       1.0  7.000000
  PRSP     2002-06-01         1.0       2.0  0.500000
  AVTA     2002-09-13       

: 